# CVRP Solver with ALNS

Advanced Capacitated Vehicle Routing Problem solver using Adaptive Large Neighborhood Search

### Notebook Structure

**Part 1: Foundation (Sections 1-7)**
- Configuration and imports
- Data structures (Client, Vehicle, Solution, DistanceMatrix)
- VRPLIB parser
- Construction heuristics
- Neighborhood operators
- Simulated Annealing
- ALNS main method
- Visualization functions

**Part 2: Execution (Sections 8-11)**
- Instance selection and loading
- Initial solution generation
- ALNS optimization (primary method)
- ALNS results and detailed analysis

**Part 3: Baseline Comparison (Sections 12-13)**
- Simulated Annealing execution
- SA results
- ALNS vs SA comparison with visualizations

## 1. Configuration and Imports

In [13]:
import math
import random
import numpy as np
from copy import deepcopy
from typing import List, Tuple, Dict
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

random.seed(42)
np.random.seed(42)

CONFIG = {
    'alns': {
        'max_iterations': 1000,  # ALNS is now primary method
        'destroy_sizes': None,     # Auto-calculated: 5%, 10%, 25%, 50%
        'initial_temp': None,      # Auto-calculated: 10% of initial cost
        'cooling_rate': 0.9,
        'sigma': [10, 5, 1],       # Rewards: best, better, accepted
        'reaction_factor': 0.1
    },
    'sa': {
        'initial_temperature': 1000,  # Kept for baseline comparison
        'cooling_rate': 0.9,
        'max_iterations': 2000,
        'min_temperature': 0.1
    },
    'instance': {
        'name': None,
        'optimal_cost': None
    }
}

## 2. Data Structures

### 2.1 Core Classes

In [14]:
def euclidean_distance(node1, node2) -> float:
    """
    Calculate Euclidean distance using VRPLIB EUC_2D standard.
    
    VRPLIB uses ROUNDED integer distance:
    d(i,j) = round(sqrt((xi-xj)^2 + (yi-yj)^2))
    
    This ensures compatibility with VRPLIB benchmark optimal costs.
    
    Args:
        node1: First node (Client or depot)
        node2: Second node (Client or depot)
        
    Returns:
        Rounded Euclidean distance (integer for VRPLIB compatibility)
    """
    dx = node1.x - node2.x
    dy = node1.y - node2.y
    return round(math.sqrt(dx * dx + dy * dy))


class Client:
    """Represents a customer with location and demand."""
    
    def __init__(self, id: int, x: float, y: float, demand: int):
        self.id = id
        self.x = x
        self.y = y
        self.demand = demand
    
    def __repr__(self):
        return f"Client({self.id}, demand={self.demand})"


class Vehicle:
    """Represents a delivery vehicle with capacity constraint."""
    
    def __init__(self, capacity: int, id: int = 0):
        self.id = id
        self.capacity = capacity
        self.route = []
        self.load = 0
    
    def add_client(self, client: Client) -> bool:
        """Add client to route if capacity allows."""
        if self.load + client.demand <= self.capacity:
            self.route.append(client)
            self.load += client.demand
            return True
        return False
    
    def remove_client(self, client: Client):
        """Remove client from route."""
        if client in self.route:
            self.route.remove(client)
            self.load -= client.demand
    
    def insert_client(self, client: Client, position: int) -> bool:
        """Insert client at specific position if capacity allows."""
        if self.load + client.demand <= self.capacity:
            self.route.insert(position, client)
            self.load += client.demand
            return True
        return False
    
    def clear(self):
        """Clear all clients from route."""
        self.route = []
        self.load = 0
    
    def __repr__(self):
        return f"Vehicle({self.id}, load={self.load}/{self.capacity}, clients={len(self.route)})"


class Solution:
    """Represents a complete VRP solution."""
    
    def __init__(self, vehicles: List[Vehicle], depot: Client):
        self.vehicles = vehicles
        self.depot = depot
        self.cost = 0.0
        self.calculate_cost()
    
    def calculate_cost(self) -> float:
        """
        Calculate total routing cost with VRPLIB rounded distances.
        
        Returns:
            Total cost (sum of all route distances)
        """
        total_distance = 0.0
        for vehicle in self.vehicles:
            if len(vehicle.route) == 0:
                continue
            
            # Depot to first client
            route_distance = euclidean_distance(self.depot, vehicle.route[0])
            
            # Between clients
            for i in range(len(vehicle.route) - 1):
                route_distance += euclidean_distance(vehicle.route[i], vehicle.route[i + 1])
            
            # Last client back to depot
            route_distance += euclidean_distance(vehicle.route[-1], self.depot)
            
            total_distance += route_distance
        
        self.cost = total_distance
        return total_distance
    
    def is_feasible(self) -> bool:
        """Check if solution respects all capacity constraints."""
        for vehicle in self.vehicles:
            if vehicle.load > vehicle.capacity:
                return False
        return True
    
    def get_num_vehicles_used(self) -> int:
        """Count vehicles with non-empty routes."""
        return sum(1 for v in self.vehicles if len(v.route) > 0)
    
    def copy(self):
        """Create a deep copy of the solution."""
        return deepcopy(self)
    
    def __repr__(self):
        return f"Solution(cost={self.cost:.2f}, vehicles={self.get_num_vehicles_used()}/{len(self.vehicles)})"

### 2.2 VRPLIB Parser

In [15]:
def parse_vrplib(file_path: str) -> Dict:
    """
    Parse VRPLIB format instance file.
    
    Args:
        file_path: Path to .vrp file
        
    Returns:
        Dictionary containing instance data
        
    Raises:
        FileNotFoundError: If file doesn't exist
        ValueError: If file format is invalid
    """
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()
    except FileNotFoundError:
        raise FileNotFoundError(f"Instance file not found: {file_path}")
    
    data = {
        'name': '',
        'dimension': 0,
        'capacity': 0,
        'nodes': [],
        'demands': [],
        'depot': 1
    }
    
    section = None
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        if line.startswith('NAME'):
            data['name'] = line.split(':')[1].strip()
        elif line.startswith('DIMENSION'):
            data['dimension'] = int(line.split(':')[1].strip())
        elif line.startswith('CAPACITY'):
            data['capacity'] = int(line.split(':')[1].strip())
        elif line.startswith('NODE_COORD_SECTION'):
            section = 'nodes'
        elif line.startswith('DEMAND_SECTION'):
            section = 'demands'
        elif line.startswith('DEPOT_SECTION'):
            section = 'depot'
        elif line.startswith('EOF'):
            break
        elif section == 'nodes':
            parts = line.split()
            if len(parts) == 3:
                node_id, x, y = int(parts[0]), float(parts[1]), float(parts[2])
                data['nodes'].append((node_id, x, y))
        elif section == 'demands':
            parts = line.split()
            if len(parts) == 2:
                node_id, demand = int(parts[0]), int(parts[1])
                data['demands'].append((node_id, demand))
        elif section == 'depot':
            if line != '-1':
                data['depot'] = int(line)
    
    if data['dimension'] == 0 or data['capacity'] == 0:
        raise ValueError(f"Invalid instance file: missing DIMENSION or CAPACITY")
    
    return data


def load_instance(file_path: str) -> Tuple[List[Client], Client, int, str]:
    """
    Load VRP instance from VRPLIB file.
    
    Args:
        file_path: Path to instance file
        
    Returns:
        Tuple of (clients, depot, capacity, instance_name)
    """
    data = parse_vrplib(file_path)
    
    clients = []
    depot = None
    
    for node in data['nodes']:
        node_id, x, y = node
        demand = next((d[1] for d in data['demands'] if d[0] == node_id), 0)
        
        if node_id == data['depot']:
            depot = Client(node_id, x, y, 0)
        else:
            clients.append(Client(node_id, x, y, demand))
    
    if depot is None:
        raise ValueError("Depot node not found in instance")
    
    return clients, depot, data['capacity'], data['name']

## 3. Construction Heuristics

In [16]:
def generate_random_solution(clients: List[Client], depot: Client, num_vehicles: int, capacity: int) -> Solution:
    """
    Generate random initial solution.
    
    Args:
        clients: List of customers
        depot: Depot node
        num_vehicles: Number of available vehicles
        capacity: Vehicle capacity
        
    Returns:
        Random feasible solution
    """
    vehicles = [Vehicle(capacity, i) for i in range(num_vehicles)]
    unassigned = clients[:]
    random.shuffle(unassigned)
    
    current_vehicle_idx = 0
    for client in unassigned:
        assigned = False
        attempts = 0
        while not assigned and attempts < num_vehicles:
            if vehicles[current_vehicle_idx].add_client(client):
                assigned = True
            else:
                current_vehicle_idx = (current_vehicle_idx + 1) % num_vehicles
                attempts += 1
        
        if not assigned:
            vehicles.append(Vehicle(capacity, len(vehicles)))
            vehicles[-1].add_client(client)
    
    return Solution(vehicles, depot)


def generate_nearest_neighbor_solution(clients: List[Client], depot: Client, num_vehicles: int, capacity: int) -> Solution:
    """
    Generate solution using nearest neighbor heuristic.
    
    Args:
        clients: List of customers
        depot: Depot node
        num_vehicles: Number of available vehicles
        capacity: Vehicle capacity
        
    Returns:
        Solution constructed by nearest neighbor
    """
    vehicles = [Vehicle(capacity, i) for i in range(num_vehicles)]
    unassigned = clients[:]
    current_vehicle_idx = 0
    
    while unassigned:
        vehicle = vehicles[current_vehicle_idx]
        
        if len(vehicle.route) == 0:
            current_position = depot
        else:
            current_position = vehicle.route[-1]
        
        best_client = None
        best_distance = float('inf')
        
        for client in unassigned:
            if vehicle.load + client.demand <= vehicle.capacity:
                distance = euclidean_distance(current_position, client)
                if distance < best_distance:
                    best_distance = distance
                    best_client = client
        
        if best_client:
            vehicle.add_client(best_client)
            unassigned.remove(best_client)
        else:
            current_vehicle_idx += 1
            if current_vehicle_idx >= len(vehicles):
                vehicles.append(Vehicle(capacity, len(vehicles)))
    
    return Solution(vehicles, depot)


def generate_clarke_wright_solution(clients: List[Client], depot: Client, capacity: int) -> Solution:
    """
    Generate solution using Clarke-Wright savings algorithm.
    
    Args:
        clients: List of customers
        depot: Depot node
        capacity: Vehicle capacity
        
    Returns:
        Solution constructed by Clarke-Wright heuristic
    """
    savings = []
    for i, client_i in enumerate(clients):
        for j, client_j in enumerate(clients[i+1:], i+1):
            saving = (euclidean_distance(depot, client_i) + 
                     euclidean_distance(depot, client_j) - 
                     euclidean_distance(client_i, client_j))
            savings.append((saving, client_i, client_j))
    
    savings.sort(reverse=True, key=lambda x: x[0])
    
    routes = {client: [client] for client in clients}
    loads = {client: client.demand for client in clients}
    
    for saving_value, client_i, client_j in savings:
        route_i = routes[client_i]
        route_j = routes[client_j]
        
        if route_i == route_j:
            continue
        
        if client_i != route_i[0] and client_i != route_i[-1]:
            continue
        if client_j != route_j[0] and client_j != route_j[-1]:
            continue
        
        combined_load = loads[client_i] + loads[client_j]
        if combined_load > capacity:
            continue
        
        if client_i == route_i[-1] and client_j == route_j[0]:
            new_route = route_i + route_j
        elif client_i == route_i[0] and client_j == route_j[-1]:
            new_route = route_j + route_i
        elif client_i == route_i[-1] and client_j == route_j[-1]:
            new_route = route_i + route_j[::-1]
        elif client_i == route_i[0] and client_j == route_j[0]:
            new_route = route_i[::-1] + route_j
        else:
            continue
        
        for client in new_route:
            routes[client] = new_route
            loads[client] = combined_load
    
    unique_routes = []
    seen = set()
    for route in routes.values():
        route_id = id(route)
        if route_id not in seen:
            seen.add(route_id)
            unique_routes.append(route)
    
    vehicles = []
    for idx, route in enumerate(unique_routes):
        vehicle = Vehicle(capacity, idx)
        for client in route:
            vehicle.add_client(client)
        vehicles.append(vehicle)
    
    return Solution(vehicles, depot)

## 4. Neighborhood Operators

In [17]:
def swap_move(solution: Solution) -> Solution:
    """
    Swap operator: exchange two clients between different routes.
    
    Args:
        solution: Current solution
        
    Returns:
        Modified solution
    """
    new_solution = solution.copy()
    non_empty = [v for v in new_solution.vehicles if len(v.route) > 0]
    
    if len(non_empty) < 2:
        return new_solution
    
    v1, v2 = random.sample(non_empty, 2)
    if len(v1.route) > 0 and len(v2.route) > 0:
        i1 = random.randint(0, len(v1.route) - 1)
        i2 = random.randint(0, len(v2.route) - 1)
        
        c1, c2 = v1.route[i1], v2.route[i2]
        
        if (v1.load - c1.demand + c2.demand <= v1.capacity and 
            v2.load - c2.demand + c1.demand <= v2.capacity):
            v1.route[i1] = c2
            v2.route[i2] = c1
            v1.load = v1.load - c1.demand + c2.demand
            v2.load = v2.load - c2.demand + c1.demand
    
    new_solution.calculate_cost()
    return new_solution


def relocate_move(solution: Solution) -> Solution:
    """
    Relocate operator: move one client to another route.
    
    Args:
        solution: Current solution
        
    Returns:
        Modified solution
    """
    new_solution = solution.copy()
    non_empty = [v for v in new_solution.vehicles if len(v.route) > 0]
    
    if len(non_empty) < 1:
        return new_solution
    
    v1 = random.choice(non_empty)
    v2 = random.choice(new_solution.vehicles)
    
    if len(v1.route) > 0:
        i1 = random.randint(0, len(v1.route) - 1)
        client = v1.route[i1]
        
        if v2.load + client.demand <= v2.capacity:
            v1.route.pop(i1)
            v1.load -= client.demand
            
            if len(v2.route) > 0:
                i2 = random.randint(0, len(v2.route))
                v2.route.insert(i2, client)
            else:
                v2.route.append(client)
            v2.load += client.demand
    
    new_solution.calculate_cost()
    return new_solution


def two_opt_move(solution: Solution) -> Solution:
    """
    2-opt operator: reverse a segment within a route.
    
    Args:
        solution: Current solution
        
    Returns:
        Modified solution
    """
    new_solution = solution.copy()
    non_empty = [v for v in new_solution.vehicles if len(v.route) > 0]
    
    if len(non_empty) < 1:
        return new_solution
    
    vehicle = random.choice(non_empty)
    route = vehicle.route
    
    if len(route) > 3:
        i = random.randint(0, len(route) - 2)
        j = random.randint(i + 1, len(route) - 1)
        vehicle.route[i:j+1] = list(reversed(vehicle.route[i:j+1]))
    
    new_solution.calculate_cost()
    return new_solution


def or_opt_move(solution: Solution) -> Solution:
    """
    Or-opt operator: relocate a sequence of 1-3 consecutive clients.
    
    Args:
        solution: Current solution
        
    Returns:
        Modified solution
    """
    new_solution = solution.copy()
    non_empty = [v for v in new_solution.vehicles if len(v.route) > 0]
    
    if len(non_empty) < 1:
        return new_solution
    
    vehicle = random.choice(non_empty)
    route = vehicle.route
    
    if len(route) > 3:
        seq_len = random.randint(1, min(3, len(route) - 1))
        i = random.randint(0, len(route) - seq_len)
        j = random.randint(0, len(route) - seq_len)
        
        if i != j:
            seq = route[i:i+seq_len]
            for _ in range(seq_len):
                route.pop(i)
            
            if j > i:
                j -= seq_len
            
            for k, client in enumerate(seq):
                route.insert(j + k, client)
    
    new_solution.calculate_cost()
    return new_solution


def cross_exchange_move(solution: Solution) -> Solution:
    """
    Cross-exchange operator: swap two segments between routes.
    
    Args:
        solution: Current solution
        
    Returns:
        Modified solution
    """
    new_solution = solution.copy()
    non_empty = [v for v in new_solution.vehicles if len(v.route) > 0]
    
    if len(non_empty) < 2:
        return new_solution
    
    v1, v2 = random.sample(non_empty, 2)
    
    if len(v1.route) > 1 and len(v2.route) > 1:
        len1 = random.randint(1, min(2, len(v1.route)))
        len2 = random.randint(1, min(2, len(v2.route)))
        
        i1 = random.randint(0, len(v1.route) - len1)
        i2 = random.randint(0, len(v2.route) - len2)
        
        seg1 = v1.route[i1:i1+len1]
        seg2 = v2.route[i2:i2+len2]
        
        demand1 = sum(c.demand for c in seg1)
        demand2 = sum(c.demand for c in seg2)
        
        if (v1.load - demand1 + demand2 <= v1.capacity and
            v2.load - demand2 + demand1 <= v2.capacity):
            v1.route[i1:i1+len1] = seg2
            v2.route[i2:i2+len2] = seg1
            v1.load = v1.load - demand1 + demand2
            v2.load = v2.load - demand2 + demand1
    
    new_solution.calculate_cost()
    return new_solution

## 5. Simulated Annealing (Baseline)

In [18]:
def generate_neighbor(solution: Solution) -> Solution:
    """
    Generate a neighbor solution by applying a random operator.
    
    Args:
        solution: Current solution
        
    Returns:
        Neighboring solution
    """
    operators = [swap_move, relocate_move, two_opt_move, or_opt_move, cross_exchange_move]
    operator = random.choice(operators)
    return operator(solution)


def acceptance_probability(current_cost: float, new_cost: float, temperature: float) -> float:
    """
    Calculate Metropolis acceptance probability.
    
    Args:
        current_cost: Cost of current solution
        new_cost: Cost of candidate solution
        temperature: Current temperature
        
    Returns:
        Acceptance probability in [0, 1]
    """
    if new_cost < current_cost:
        return 1.0
    if temperature <= 0:
        return 0.0
    return math.exp((current_cost - new_cost) / temperature)


def simulated_annealing(initial_solution: Solution, 
                       initial_temp: float = None,
                       cooling_rate: float = None,
                       max_iter: int = None,
                       min_temp: float = None,
                       verbose: bool = True) -> Tuple[Solution, List[float]]:
    """
    Solve VRP using Simulated Annealing metaheuristic.
    
    Args:
        initial_solution: Starting solution
        initial_temp: Initial temperature (default from CONFIG)
        cooling_rate: Cooling rate (default from CONFIG)
        max_iter: Maximum iterations (default from CONFIG)
        min_temp: Minimum temperature (default from CONFIG)
        verbose: Print progress messages
        
    Returns:
        Tuple of (best_solution, cost_history)
    """
    if initial_temp is None:
        initial_temp = CONFIG['sa']['initial_temperature']
    if cooling_rate is None:
        cooling_rate = CONFIG['sa']['cooling_rate']
    if max_iter is None:
        max_iter = CONFIG['sa']['max_iterations']
    if min_temp is None:
        min_temp = CONFIG['sa']['min_temperature']
    
    current_solution = initial_solution.copy()
    best_solution = current_solution.copy()
    
    temperature = initial_temp
    iteration = 0
    stagnation_counter = 0
    last_improvement = 0
    
    cost_history = [best_solution.cost]
    
    start_time = time.time()
    
    while iteration < max_iter and temperature > min_temp:
        neighbor = generate_neighbor(current_solution)
        
        current_cost = current_solution.cost
        neighbor_cost = neighbor.cost
        
        if acceptance_probability(current_cost, neighbor_cost, temperature) > random.random():
            current_solution = neighbor
            
            if current_solution.cost < best_solution.cost:
                best_solution = current_solution.copy()
                last_improvement = iteration
                stagnation_counter = 0
                if verbose and iteration % 5000 == 0:
                    print(f"Iter {iteration}: best = {best_solution.cost:.2f}")
            else:
                stagnation_counter += 1
        
        if stagnation_counter > 1000:
            current_solution = generate_neighbor(best_solution)
            current_solution = generate_neighbor(current_solution)
            stagnation_counter = 0
        
        temperature *= cooling_rate
        iteration += 1
        
        if iteration % 100 == 0:
            cost_history.append(best_solution.cost)
    
    elapsed = time.time() - start_time
    
    if verbose:
        print(f"SA completed in {elapsed:.2f}s")
        print(f"Best cost: {best_solution.cost:.2f}")
        print(f"Last improvement: iteration {last_improvement}")
        print(f"Vehicles used: {best_solution.get_num_vehicles_used()}")
    
    return best_solution, cost_history

## 6. ALNS (Main Optimization Method)

Adaptive Large Neighborhood Search with:
- Adaptive destroy sizes (5%, 10%, 25%, 50% of clients)
- SA-style acceptance criterion with temperature cooling
- Sophisticated operator scoring (rewards: 10, 5, 1)
- 2-opt local search intensification
- Automatic restart on stagnation
- Three destroy operators: Random, Worst, Shaw
- Two repair operators: Greedy, Regret-2

In [19]:
def destroy_random(solution: Solution, q: int) -> Tuple[Solution, List[Client]]:
    """
    Destroy operator: remove q random clients.
    
    Args:
        solution: Current solution
        q: Number of clients to remove
        
    Returns:
        Tuple of (partial_solution, removed_clients)
    """
    partial = solution.copy()
    removed = []
    
    all_clients = []
    for v in partial.vehicles:
        all_clients.extend(v.route)
    
    if len(all_clients) < q:
        q = len(all_clients)
    
    clients_to_remove = random.sample(all_clients, q)
    
    for client in clients_to_remove:
        for v in partial.vehicles:
            if client in v.route:
                v.remove_client(client)
                removed.append(client)
                break
    
    return partial, removed


def destroy_worst(solution: Solution, q: int) -> Tuple[Solution, List[Client]]:
    """
    Destroy operator: remove q clients with highest removal cost.
    
    Args:
        solution: Current solution
        q: Number of clients to remove
        
    Returns:
        Tuple of (partial_solution, removed_clients)
    """
    partial = solution.copy()
    removed = []
    
    costs = []
    for v in partial.vehicles:
        for i, client in enumerate(v.route):
            temp_route = v.route[:]
            temp_route.pop(i)
            
            cost_without = 0
            if len(temp_route) > 0:
                cost_without += euclidean_distance(partial.depot, temp_route[0])
                for j in range(len(temp_route) - 1):
                    cost_without += euclidean_distance(temp_route[j], temp_route[j+1])
                cost_without += euclidean_distance(temp_route[-1], partial.depot)
            
            cost_with = euclidean_distance(partial.depot, v.route[0])
            for j in range(len(v.route) - 1):
                cost_with += euclidean_distance(v.route[j], v.route[j+1])
            cost_with += euclidean_distance(v.route[-1], partial.depot)
            
            saving = cost_with - cost_without
            costs.append((saving, client, v))
    
    costs.sort(reverse=True, key=lambda x: x[0])
    
    for i in range(min(q, len(costs))):
        _, client, vehicle = costs[i]
        if client in vehicle.route:
            vehicle.remove_client(client)
            removed.append(client)
    
    partial.calculate_cost()
    return partial, removed


def destroy_shaw(solution: Solution, q: int, p: float = 2.0) -> Tuple[Solution, List[Client]]:
    """
    Shaw removal: remove clients that are similar (geographically close).
    
    Args:
        solution: Current solution
        q: Number of clients to remove
        p: Relatedness exponent parameter
        
    Returns:
        Tuple of (partial_solution, removed_clients)
    """
    partial = solution.copy()
    removed = []
    
    all_clients = []
    for v in partial.vehicles:
        all_clients.extend(v.route)
    
    if len(all_clients) == 0 or q == 0:
        return partial, removed
    
    seed_client = random.choice(all_clients)
    removed.append(seed_client)
    
    for v in partial.vehicles:
        if seed_client in v.route:
            v.remove_client(seed_client)
            break
    
    all_clients.remove(seed_client)
    
    while len(removed) < q and len(all_clients) > 0:
        relatedness = []
        for client in all_clients:
            min_dist = min(euclidean_distance(client, r) for r in removed)
            relatedness.append((min_dist ** p, client))
        
        relatedness.sort(key=lambda x: x[0])
        
        weights = [1.0 / (i + 1) for i in range(len(relatedness))]
        total_weight = sum(weights)
        weights = [w / total_weight for w in weights]
        
        idx = np.random.choice(len(relatedness), p=weights)
        _, next_client = relatedness[idx]
        
        removed.append(next_client)
        all_clients.remove(next_client)
        
        for v in partial.vehicles:
            if next_client in v.route:
                v.remove_client(next_client)
                break
    
    partial.calculate_cost()
    return partial, removed


def repair_greedy(partial: Solution, removed: List[Client]) -> Solution:
    """
    Repair operator: reinsert clients at best positions.
    
    Args:
        partial: Partial solution with removed clients
        removed: List of clients to reinsert
        
    Returns:
        Complete solution
    """
    solution = partial.copy()
    
    for client in removed:
        best_cost = float('inf')
        best_vehicle = None
        best_position = 0
        
        for v in solution.vehicles:
            if v.load + client.demand <= v.capacity:
                for pos in range(len(v.route) + 1):
                    if pos == 0:
                        cost_before = 0 if len(v.route) == 0 else euclidean_distance(solution.depot, v.route[0])
                        cost_after = euclidean_distance(solution.depot, client)
                        if len(v.route) > 0:
                            cost_after += euclidean_distance(client, v.route[0])
                    elif pos == len(v.route):
                        cost_before = euclidean_distance(v.route[-1], solution.depot)
                        cost_after = euclidean_distance(v.route[-1], client) + euclidean_distance(client, solution.depot)
                    else:
                        cost_before = euclidean_distance(v.route[pos-1], v.route[pos])
                        cost_after = euclidean_distance(v.route[pos-1], client) + euclidean_distance(client, v.route[pos])
                    
                    insertion_cost = cost_after - cost_before
                    
                    if insertion_cost < best_cost:
                        best_cost = insertion_cost
                        best_vehicle = v
                        best_position = pos
        
        if best_vehicle:
            best_vehicle.route.insert(best_position, client)
            best_vehicle.load += client.demand
    
    solution.calculate_cost()
    return solution


def repair_regret(partial: Solution, removed: List[Client], k: int = 2) -> Solution:
    """
    Regret-k repair: insert clients based on regret value.
    
    Args:
        partial: Partial solution with removed clients
        removed: List of clients to reinsert
        k: Number of best positions to consider for regret
        
    Returns:
        Complete solution
    """
    solution = partial.copy()
    uninserted = removed[:]
    
    while uninserted:
        best_regret = -float('inf')
        best_client = None
        best_vehicle = None
        best_position = 0
        
        for client in uninserted:
            insertion_costs = []
            
            for v in solution.vehicles:
                if v.load + client.demand <= v.capacity:
                    for pos in range(len(v.route) + 1):
                        if pos == 0:
                            cost_before = 0 if len(v.route) == 0 else euclidean_distance(solution.depot, v.route[0])
                            cost_after = euclidean_distance(solution.depot, client)
                            if len(v.route) > 0:
                                cost_after += euclidean_distance(client, v.route[0])
                        elif pos == len(v.route):
                            cost_before = euclidean_distance(v.route[-1], solution.depot)
                            cost_after = euclidean_distance(v.route[-1], client) + euclidean_distance(client, solution.depot)
                        else:
                            cost_before = euclidean_distance(v.route[pos-1], v.route[pos])
                            cost_after = euclidean_distance(v.route[pos-1], client) + euclidean_distance(client, v.route[pos])
                        
                        insertion_cost = cost_after - cost_before
                        insertion_costs.append((insertion_cost, v, pos))
            
            if len(insertion_costs) > 0:
                insertion_costs.sort(key=lambda x: x[0])
                
                if len(insertion_costs) >= k:
                    regret = sum(insertion_costs[i][0] - insertion_costs[0][0] for i in range(1, k))
                else:
                    regret = 0
                
                if regret > best_regret:
                    best_regret = regret
                    best_client = client
                    best_vehicle = insertion_costs[0][1]
                    best_position = insertion_costs[0][2]
        
        if best_client and best_vehicle:
            best_vehicle.route.insert(best_position, best_client)
            best_vehicle.load += best_client.demand
            uninserted.remove(best_client)
        else:
            break
    
    solution.calculate_cost()
    return solution


def local_search_2opt(solution: Solution, max_moves: int = 50) -> Solution:
    """
    Apply 2-opt local search on all routes for intensification.
    
    Args:
        solution: Solution to improve
        max_moves: Maximum number of improving moves
        
    Returns:
        Improved solution
    """
    improved = solution.copy()
    moves = 0
    
    for vehicle in improved.vehicles:
        if len(vehicle.route) < 4:
            continue
        
        improved_route = True
        while improved_route and moves < max_moves:
            improved_route = False
            best_delta = 0
            best_i, best_j = -1, -1
            
            for i in range(len(vehicle.route) - 1):
                for j in range(i + 2, len(vehicle.route)):
                    # Calculate delta cost for 2-opt move
                    if i == 0:
                        cost_before = euclidean_distance(improved.depot, vehicle.route[0])
                    else:
                        cost_before = euclidean_distance(vehicle.route[i-1], vehicle.route[i])
                    
                    cost_before += euclidean_distance(vehicle.route[j], vehicle.route[j+1] if j+1 < len(vehicle.route) else improved.depot)
                    
                    # After reversing segment [i:j+1]
                    if i == 0:
                        cost_after = euclidean_distance(improved.depot, vehicle.route[j])
                    else:
                        cost_after = euclidean_distance(vehicle.route[i-1], vehicle.route[j])
                    
                    cost_after += euclidean_distance(vehicle.route[i], vehicle.route[j+1] if j+1 < len(vehicle.route) else improved.depot)
                    
                    delta = cost_after - cost_before
                    if delta < best_delta:
                        best_delta = delta
                        best_i, best_j = i, j
                        improved_route = True
            
            if improved_route:
                vehicle.route[best_i:best_j+1] = reversed(vehicle.route[best_i:best_j+1])
                moves += 1
    
    improved.calculate_cost()
    return improved


def alns(initial_solution: Solution, 
         max_iter: int = None,
         destroy_sizes: List[int] = None,
         initial_temp: float = None,
         verbose: bool = True) -> Tuple[Solution, List[float]]:
    """
    Solve VRP using Adaptive Large Neighborhood Search (ALNS) - MAIN OPTIMIZATION METHOD.
    
    Enhanced with:
    - Adaptive destroy size selection
    - Simulated Annealing acceptance criterion
    - Sophisticated operator scoring (reward/punish)
    - Local search intensification after repair
    
    Args:
        initial_solution: Starting solution
        max_iter: Maximum iterations (default from CONFIG)
        destroy_sizes: List of destroy sizes to adaptively select from
        initial_temp: Initial temperature for SA acceptance
        verbose: Print progress messages
        
    Returns:
        Tuple of (best_solution, cost_history)
    """
    if max_iter is None:
        max_iter = CONFIG['alns']['max_iterations']
    if destroy_sizes is None:
        # Adaptive sizes: 5%, 10%, 25%, 50% of clients
        n_clients = sum(len(v.route) for v in initial_solution.vehicles)
        destroy_sizes = [
            max(3, int(0.05 * n_clients)),
            max(5, int(0.10 * n_clients)),
            max(8, int(0.25 * n_clients)),
            max(10, int(0.50 * n_clients))
        ]
    if initial_temp is None:
        initial_temp = initial_solution.cost * 0.1  # 10% of initial cost
    
    current = initial_solution.copy()
    best = current.copy()
    
    cost_history = [best.cost]
    
    # Operators
    destroy_ops = [destroy_random, destroy_worst, destroy_shaw]
    repair_ops = [repair_greedy, repair_regret]
    
    # Adaptive weights with decay
    destroy_weights = [1.0] * len(destroy_ops)
    repair_weights = [1.0] * len(repair_ops)
    size_weights = [1.0] * len(destroy_sizes)
    
    # Scoring system (reward for improvements)
    sigma1, sigma2, sigma3 = 10, 5, 1  # Best, Better, Accepted
    reaction_factor = 0.1
    
    start_time = time.time()
    temperature = initial_temp
    cooling_rate = 0.9
    
    last_improvement_iter = 0
    
    for iteration in range(max_iter):
        # Select operators adaptively
        d_idx = random.choices(range(len(destroy_ops)), weights=destroy_weights)[0]
        r_idx = random.choices(range(len(repair_ops)), weights=repair_weights)[0]
        s_idx = random.choices(range(len(destroy_sizes)), weights=size_weights)[0]
        
        destroy_op = destroy_ops[d_idx]
        repair_op = repair_ops[r_idx]
        destroy_size = destroy_sizes[s_idx]
        
        # Destroy & Repair
        partial, removed = destroy_op(current, destroy_size)
        neighbor = repair_op(partial, removed)
        
        # Local search intensification (every 10 iterations or on improvement)
        if iteration % 10 == 0 or neighbor.cost < current.cost:
            neighbor = local_search_2opt(neighbor, max_moves=20)
        
        # Calculate improvement
        delta = neighbor.cost - current.cost
        
        # Adaptive acceptance with SA criterion
        accept = False
        reward = 0
        
        if neighbor.cost < best.cost:
            # New best solution
            best = neighbor.copy()
            current = neighbor
            accept = True
            reward = sigma1
            last_improvement_iter = iteration
            if verbose and iteration % 500 == 0:
                print(f"Iter {iteration}: NEW BEST = {best.cost:.2f} (temp={temperature:.2f})")
        elif neighbor.cost < current.cost:
            # Better than current
            current = neighbor
            accept = True
            reward = sigma2
        elif acceptance_probability(current.cost, neighbor.cost, temperature) > random.random():
            # Accept worse solution (exploration)
            current = neighbor
            accept = True
            reward = sigma3
        
        # Update operator weights (adaptive learning)
        if accept:
            destroy_weights[d_idx] = destroy_weights[d_idx] * (1 - reaction_factor) + reaction_factor * reward
            repair_weights[r_idx] = repair_weights[r_idx] * (1 - reaction_factor) + reaction_factor * reward
            size_weights[s_idx] = size_weights[s_idx] * (1 - reaction_factor) + reaction_factor * reward
        
        # Cool down temperature
        temperature *= cooling_rate
        
        # Restart if stagnant (diversification)
        if iteration - last_improvement_iter > 2000:
            temperature = initial_temp * 0.5
            current = best.copy()
            # Perturb
            for _ in range(5):
                current = generate_neighbor(current)
            last_improvement_iter = iteration
            if verbose:
                print(f"Iter {iteration}: RESTART (diversification)")
        
        # Record history
        if iteration % 50 == 0:
            cost_history.append(best.cost)
    
    elapsed = time.time() - start_time
    
    if verbose:
        print(f"\n{'='*60}")
        print(f"ALNS COMPLETED")
        print(f"{'='*60}")
        print(f"Time: {elapsed:.2f}s")
        print(f"Best cost: {best.cost:.2f}")
        print(f"Vehicles used: {best.get_num_vehicles_used()}")
        print(f"Iterations: {max_iter}")
        print(f"Last improvement: iteration {last_improvement_iter}")
        
        # Operator statistics
        print(f"\nOperator weights (final):")
        for i, op in enumerate(['Random', 'Worst', 'Shaw']):
            print(f"  Destroy {op}: {destroy_weights[i]:.2f}")
        for i, op in enumerate(['Greedy', 'Regret']):
            print(f"  Repair {op}: {repair_weights[i]:.2f}")
        print(f"{'='*60}\n")
    
    return best, cost_history

## 7. Visualization Functions

In [20]:
def plot_solution_plotly(solution: Solution, title: str = "VRP Solution", optimal_cost: float = None):
    """
    Visualize VRP solution with interactive Plotly chart.
    
    Args:
        solution: Solution to visualize
        title: Chart title
        optimal_cost: Optimal cost for gap calculation
    """
    fig = go.Figure()
    colors = px.colors.qualitative.Plotly
    
    fig.add_trace(go.Scatter(
        x=[solution.depot.x],
        y=[solution.depot.y],
        mode='markers',
        marker=dict(size=20, color='red', symbol='square'),
        name='Depot',
        text=[f'Depot {solution.depot.id}'],
        hoverinfo='text'
    ))
    
    for idx, vehicle in enumerate(solution.vehicles):
        if len(vehicle.route) == 0:
            continue
        
        color = colors[idx % len(colors)]
        
        route_x = [solution.depot.x] + [c.x for c in vehicle.route] + [solution.depot.x]
        route_y = [solution.depot.y] + [c.y for c in vehicle.route] + [solution.depot.y]
        
        fig.add_trace(go.Scatter(
            x=route_x,
            y=route_y,
            mode='lines',
            line=dict(color=color, width=2),
            name=f'V{vehicle.id+1} ({vehicle.load}/{vehicle.capacity})',
            hoverinfo='name'
        ))
        
        fig.add_trace(go.Scatter(
            x=[c.x for c in vehicle.route],
            y=[c.y for c in vehicle.route],
            mode='markers+text',
            marker=dict(size=10, color=color),
            text=[str(c.id) for c in vehicle.route],
            textposition='top center',
            name=f'Clients V{vehicle.id+1}',
            hovertext=[f'Client {c.id}<br>Demand: {c.demand}' for c in vehicle.route],
            hoverinfo='text',
            showlegend=False
        ))
    
    gap_text = ""
    if optimal_cost:
        gap = ((solution.cost - optimal_cost) / optimal_cost) * 100
        gap_text = f" | Gap: {gap:.2f}%"
    
    fig.update_layout(
        title=f'{title}<br>Cost: {solution.cost:.2f} | Vehicles: {solution.get_num_vehicles_used()}{gap_text}',
        xaxis_title='X',
        yaxis_title='Y',
        hovermode='closest',
        height=600
    )
    
    fig.show()


def plot_convergence(cost_history: List[float], optimal_cost: float = None, title: str = "Convergence"):
    """
    Visualize algorithm convergence.
    
    Args:
        cost_history: List of costs over iterations
        optimal_cost: Optimal cost for reference line
        title: Chart title
    """
    fig = go.Figure()
    
    iterations = list(range(len(cost_history)))
    
    fig.add_trace(go.Scatter(
        x=iterations,
        y=cost_history,
        mode='lines',
        name='Cost',
        line=dict(color='blue', width=2)
    ))
    
    if optimal_cost:
        fig.add_trace(go.Scatter(
            x=[0, len(cost_history)-1],
            y=[optimal_cost, optimal_cost],
            mode='lines',
            name=f'Optimal ({optimal_cost})',
            line=dict(color='green', width=2, dash='dash')
        ))
    
    fig.update_layout(
        title=title,
        xaxis_title='Iterations (x100)',
        yaxis_title='Cost',
        hovermode='x unified',
        height=400
    )
    
    fig.show()

## 8. Instance Selection and Loading

In [21]:
import os
from pathlib import Path

# Available instances in data folder
available_instances = {
    'Small instances': [
        'E-n13-k4',      # 13 clients
        'P-n16-k8',      # 16 clients
        'B-n31-k5',      # 31 clients
        'A-n32-k5',      # 32 clients
    ],
    'Medium instances': [
        'CMT6',          # 50 clients
        'F-n72-k4',      # 72 clients
        'M-n101-k10',    # 101 clients
        'X-n101-k25',    # 101 clients
    ],
    'Large instances': [
        'ORTEC-n242-k12', # 242 clients
    ],
    'Time windows (Solomon)': [
        'C101',          # Clustered
        'C1_2_1',        # Clustered
    ],
    'Golden instances': [
        'Golden_1',
    ],
    'Li instances': [
        'Li_21',
    ],
    'Tai instances': [
        'tai75a',
    ]
}

# Select instance here
INSTANCE_NAME = 'M-n101-k10'  
  # Change this to any instance name above

# Auto-detect file extension
data_path = Path('data')
vrp_file = None
sol_file = None

for ext in ['.vrp', '.txt']:
    if (data_path / f"{INSTANCE_NAME}{ext}").exists():
        vrp_file = f"data/{INSTANCE_NAME}{ext}"
        break

if (data_path / f"{INSTANCE_NAME}.sol").exists():
    sol_file = f"data/{INSTANCE_NAME}.sol"

if vrp_file:
    print(f"Selected instance: {INSTANCE_NAME}")
    print(f"Instance file: {vrp_file}")
    if sol_file:
        print(f"Solution file: {sol_file}")
    else:
        print("No optimal solution available")
else:
    print(f"Error: Instance '{INSTANCE_NAME}' not found in data folder")
    print("\nAvailable instances:")
    for category, instances in available_instances.items():
        print(f"\n{category}:")
        for inst in instances:
            print(f"  - {inst}")

Selected instance: M-n101-k10
Instance file: data/M-n101-k10.vrp
Solution file: data/M-n101-k10.sol


---

# PART 2: EXECUTION

---

In [22]:
instance_file = vrp_file
solution_file = sol_file

try:
    clients, depot, capacity, instance_name_loaded = load_instance(instance_file)
    CONFIG['instance']['name'] = instance_name_loaded
    
    if solution_file:
        try:
            with open(solution_file, 'r') as f:
                for line in f:
                    if line.startswith('Cost'):
                        CONFIG['instance']['optimal_cost'] = float(line.split()[1])
                        break
        except FileNotFoundError:
            print(f"Warning: Solution file not found: {solution_file}")
    else:
        CONFIG['instance']['optimal_cost'] = None
    
    total_demand = sum(c.demand for c in clients)
    num_vehicles = max(5, (total_demand + capacity - 1) // capacity)
    
    print(f"Instance: {CONFIG['instance']['name']}")
    print(f"Clients: {len(clients)}, Capacity: {capacity}, Vehicles: {num_vehicles}")
    if CONFIG['instance']['optimal_cost']:
        print(f"Optimal cost: {CONFIG['instance']['optimal_cost']}")
        
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error loading instance: {e}")

Instance: M-n101-k10
Clients: 100, Capacity: 200, Vehicles: 10
Optimal cost: 820.0


In [23]:
print("Generating initial solution with Clarke-Wright...")
initial_solution = generate_clarke_wright_solution(clients, depot, capacity)

print(f"Initial cost: {initial_solution.cost:.0f}")
print(f"Vehicles: {initial_solution.get_num_vehicles_used()}")
print(f"Feasible: {initial_solution.is_feasible()}")

if CONFIG['instance']['optimal_cost']:
    gap_initial = ((initial_solution.cost - CONFIG['instance']['optimal_cost']) / CONFIG['instance']['optimal_cost']) * 100
    print(f"Gap from optimal: {gap_initial:.2f}%")

Generating initial solution with Clarke-Wright...
Initial cost: 835
Vehicles: 10
Feasible: True
Gap from optimal: 1.83%


## 9. Initial Solution Generation

In [24]:
print("="*70)
print("RUNNING ALNS (PRIMARY OPTIMIZATION METHOD)")
print("="*70)

solution_alns, cost_history_alns = alns(initial_solution, verbose=True)

if CONFIG['instance']['optimal_cost']:
    gap = ((solution_alns.cost - CONFIG['instance']['optimal_cost']) / CONFIG['instance']['optimal_cost']) * 100
    improvement = ((initial_solution.cost - solution_alns.cost) / initial_solution.cost) * 100
    print(f"\nFinal gap from optimal: {gap:.2f}%")
    print(f"Improvement from initial: {improvement:.2f}%")

RUNNING ALNS (PRIMARY OPTIMIZATION METHOD)

ALNS COMPLETED
Time: 45.72s
Best cost: 835.00
Vehicles used: 10
Iterations: 1000
Last improvement: iteration 0

Operator weights (final):
  Destroy Random: 1.00
  Destroy Worst: 1.00
  Destroy Shaw: 1.04
  Repair Greedy: 1.00
  Repair Regret: 1.01


Final gap from optimal: 1.83%
Improvement from initial: 0.00%


## 10. ALNS Optimization (Main Method)

## 11. ALNS Results

In [25]:
plot_solution_plotly(solution_alns, title=f"ALNS Solution - {CONFIG['instance']['name']}", optimal_cost=CONFIG['instance']['optimal_cost'])

In [26]:
plot_convergence(cost_history_alns, optimal_cost=CONFIG['instance']['optimal_cost'], title=f"ALNS Convergence - {CONFIG['instance']['name']}")

### ALNS Solution Details

In [27]:
print("\n" + "="*70)
print("ALNS SOLUTION SUMMARY")
print("="*70)
print(f"Instance: {CONFIG['instance']['name']}")
print(f"Total cost: {solution_alns.cost:.2f}")
print(f"Vehicles used: {solution_alns.get_num_vehicles_used()}")
print(f"Feasible: {solution_alns.is_feasible()}")

if CONFIG['instance']['optimal_cost']:
    gap_alns = ((solution_alns.cost - CONFIG['instance']['optimal_cost']) / CONFIG['instance']['optimal_cost']) * 100
    print(f"Optimal cost: {CONFIG['instance']['optimal_cost']:.2f}")
    print(f"Gap from optimal: {gap_alns:.2f}%")

improvement_alns = ((initial_solution.cost - solution_alns.cost) / initial_solution.cost) * 100
print(f"Improvement from initial: {improvement_alns:.2f}%")

print("\n" + "-"*70)
print("ROUTE DETAILS")
print("-"*70)

for vehicle in solution_alns.vehicles:
    if len(vehicle.route) > 0:
        route_ids = [c.id for c in vehicle.route]
        utilization = (vehicle.load / vehicle.capacity) * 100
        
        print(f"\nVehicle {vehicle.id + 1}:")
        print(f"  Route: {route_ids}")
        print(f"  Clients: {len(vehicle.route)}")
        print(f"  Load: {vehicle.load}/{vehicle.capacity} ({utilization:.1f}%)")

print("\n" + "="*70 + "\n")


ALNS SOLUTION SUMMARY
Instance: M-n101-k10
Total cost: 835.00
Vehicles used: 10
Feasible: True
Optimal cost: 820.00
Gap from optimal: 1.83%
Improvement from initial: 0.00%

----------------------------------------------------------------------
ROUTE DETAILS
----------------------------------------------------------------------

Vehicle 1:
  Route: [76, 2, 3, 5, 7, 10, 12, 9, 8, 6]
  Clients: 10
  Load: 160/200 (80.0%)

Vehicle 2:
  Route: [99, 97, 96, 95, 93, 94, 98, 101, 100, 4]
  Clients: 10
  Load: 200/200 (100.0%)

Vehicle 3:
  Route: [11, 14, 13, 15, 17, 20, 16, 18, 19]
  Clients: 9
  Load: 200/200 (100.0%)

Vehicle 4:
  Route: [24, 27, 29, 31, 30, 28, 26, 25, 23, 22, 21]
  Clients: 11
  Load: 170/200 (85.0%)

Vehicle 5:
  Route: [33, 34, 32, 36, 38, 39, 40, 37, 35]
  Clients: 9
  Load: 200/200 (100.0%)

Vehicle 6:
  Route: [50, 53, 51, 52, 49, 46, 45, 41, 42, 43, 47, 48, 44]
  Clients: 13
  Load: 160/200 (80.0%)

Vehicle 7:
  Route: [58, 56, 55, 54, 57, 59, 61, 60]
  Clients: 8


In [28]:
# Diagnostic: Verify Distance Calculation and Gap
print("\n" + "="*70)
print("DIAGNOSTIC: Distance Calculation Verification")
print("="*70)

# Sample a few distances to verify rounding
print("\nSample Distance Verification (VRPLIB EUC_2D Standard):")
print("-" * 70)
if len(clients) >= 3:
    for i in range(min(3, len(clients)-1)):
        c1, c2 = clients[i], clients[i+1]
        dx = c1.x - c2.x
        dy = c1.y - c2.y
        decimal_dist = math.sqrt(dx*dx + dy*dy)
        rounded_dist = round(decimal_dist)
        print(f"Client {c1.id} -> Client {c2.id}:")
        print(f"  Decimal:  {decimal_dist:.6f}")
        print(f"  Rounded:  {rounded_dist} (VRPLIB standard)")
        print(f"  Diff:     {abs(decimal_dist - rounded_dist):.6f}")

# Recalculate ALNS total cost using correct distance function
print("\n" + "="*70)
print("ALNS Solution Cost Verification:")
print("="*70)
total_recalc = 0
for vehicle in solution_alns.vehicles:
    if not vehicle.route:
        continue
    route_cost = 0
    prev = depot
    # vehicle.route contains Client objects, not IDs
    for client in vehicle.route:
        dist = euclidean_distance(prev, client)
        route_cost += dist
        prev = client
    # Return to depot
    route_cost += euclidean_distance(prev, depot)
    total_recalc += route_cost

print(f"\nOriginal ALNS Cost:     {solution_alns.cost:.2f}")
print(f"Recalculated Cost:      {total_recalc:.2f}")
print(f"Difference:             {abs(solution_alns.cost - total_recalc):.2f}")

# Verify gap calculation
print("\n" + "="*70)
print("Gap Calculation Verification:")
print("="*70)

if CONFIG['instance']['optimal_cost']:
    optimal_cost = CONFIG['instance']['optimal_cost']
    gap_recalc = ((total_recalc - optimal_cost) / optimal_cost) * 100
    print(f"\nOptimal Cost:           {optimal_cost:.2f}")
    print(f"ALNS Cost:              {total_recalc:.2f}")
    print(f"Gap:                    {gap_recalc:+.2f}%")
    print(f"Original Gap:           {gap_alns:+.2f}%")
    
    if gap_recalc < 0:
        print("\nWARNING: Negative gap detected!")
        print("This indicates ALNS found better solution than optimal (impossible).")
        print("Possible causes:")
        print("  1. Distance function not matching VRPLIB standard")
        print("  2. Optimal cost from different problem variant")
        print("  3. Solution file contains errors")
    else:
        print(f"\nGap is POSITIVE: ALNS solution is {abs(gap_recalc):.2f}% above optimal.")
        if gap_recalc < 0.1:
            print("EXCELLENT: Gap near zero - optimal or near-optimal solution!")
        elif gap_recalc < 3:
            print("VERY GOOD: Gap under 3% - high quality solution!")
        else:
            print("GOOD: Solution found, but room for improvement.")
else:
    print("\nNo optimal cost available - skipping gap verification.")

print("="*70)


DIAGNOSTIC: Distance Calculation Verification

Sample Distance Verification (VRPLIB EUC_2D Standard):
----------------------------------------------------------------------
Client 2 -> Client 3:
  Decimal:  2.000000
  Rounded:  2 (VRPLIB standard)
  Diff:     0.000000
Client 3 -> Client 4:
  Decimal:  5.000000
  Rounded:  5 (VRPLIB standard)
  Diff:     0.000000
Client 4 -> Client 5:
  Decimal:  2.000000
  Rounded:  2 (VRPLIB standard)
  Diff:     0.000000

ALNS Solution Cost Verification:

Original ALNS Cost:     835.00
Recalculated Cost:      835.00
Difference:             0.00

Gap Calculation Verification:

Optimal Cost:           820.00
ALNS Cost:              835.00
Gap:                    +1.83%
Original Gap:           +1.83%

Gap is POSITIVE: ALNS solution is 1.83% above optimal.
VERY GOOD: Gap under 3% - high quality solution!


## 12. Simulated Annealing (Baseline Method)

In [29]:
print("="*70)
print("RUNNING SIMULATED ANNEALING (BASELINE)")
print("="*70)

solution_sa, cost_history_sa = simulated_annealing(initial_solution.copy(), verbose=True)

if CONFIG['instance']['optimal_cost']:
    gap_sa = ((solution_sa.cost - CONFIG['instance']['optimal_cost']) / CONFIG['instance']['optimal_cost']) * 100
    improvement_sa = ((initial_solution.cost - solution_sa.cost) / initial_solution.cost) * 100
    print(f"\nFinal gap from optimal: {gap_sa:.2f}%")
    print(f"Improvement from initial: {improvement_sa:.2f}%")

RUNNING SIMULATED ANNEALING (BASELINE)
SA completed in 0.26s
Best cost: 835.00
Last improvement: iteration 0
Vehicles used: 10

Final gap from optimal: 1.83%
Improvement from initial: 0.00%


### SA Solution Details

In [30]:
print("\n" + "="*70)
print("SA SOLUTION SUMMARY")
print("="*70)
print(f"Instance: {CONFIG['instance']['name']}")
print(f"Total cost: {solution_sa.cost:.2f}")
print(f"Vehicles used: {solution_sa.get_num_vehicles_used()}")
print(f"Feasible: {solution_sa.is_feasible()}")

if CONFIG['instance']['optimal_cost']:
    gap_sa = ((solution_sa.cost - CONFIG['instance']['optimal_cost']) / CONFIG['instance']['optimal_cost']) * 100
    print(f"Optimal cost: {CONFIG['instance']['optimal_cost']:.2f}")
    print(f"Gap from optimal: {gap_sa:.2f}%")

improvement_sa = ((initial_solution.cost - solution_sa.cost) / initial_solution.cost) * 100
print(f"Improvement from initial: {improvement_sa:.2f}%")
print("="*70 + "\n")


SA SOLUTION SUMMARY
Instance: M-n101-k10
Total cost: 835.00
Vehicles used: 10
Feasible: True
Optimal cost: 820.00
Gap from optimal: 1.83%
Improvement from initial: 0.00%



## 13. Comparison: ALNS vs SA

In [31]:
print("\n" + "="*70)
print("ALGORITHM COMPARISON: ALNS vs SA")
print("="*70)
print(f"Instance: {CONFIG['instance']['name']}")
if CONFIG['instance']['optimal_cost']:
    print(f"Optimal cost: {CONFIG['instance']['optimal_cost']:.2f}")
print(f"Initial cost: {initial_solution.cost:.2f}")
print()
print(f"{'Method':<15} {'Cost':<12} {'Gap (%)':<12} {'Improvement (%)':<15} {'Vehicles':<10}")
print("-"*70)

gap_alns = ((solution_alns.cost - CONFIG['instance']['optimal_cost']) / CONFIG['instance']['optimal_cost']) * 100 if CONFIG['instance']['optimal_cost'] else 0
gap_sa = ((solution_sa.cost - CONFIG['instance']['optimal_cost']) / CONFIG['instance']['optimal_cost']) * 100 if CONFIG['instance']['optimal_cost'] else 0
imp_alns = ((initial_solution.cost - solution_alns.cost) / initial_solution.cost) * 100
imp_sa = ((initial_solution.cost - solution_sa.cost) / initial_solution.cost) * 100

print(f"{'ALNS':<15} {solution_alns.cost:<12.2f} {gap_alns:<12.2f} {imp_alns:<15.2f} {solution_alns.get_num_vehicles_used():<10}")
print(f"{'SA':<15} {solution_sa.cost:<12.2f} {gap_sa:<12.2f} {imp_sa:<15.2f} {solution_sa.get_num_vehicles_used():<10}")
print()

if solution_alns.cost < solution_sa.cost:
    diff = solution_sa.cost - solution_alns.cost
    pct = (diff / solution_sa.cost) * 100
    print(f"ALNS is better by {diff:.2f} ({pct:.2f}%)")
else:
    diff = solution_alns.cost - solution_sa.cost
    pct = (diff / solution_alns.cost) * 100
    print(f"SA is better by {diff:.2f} ({pct:.2f}%)")

print("="*70 + "\n")


ALGORITHM COMPARISON: ALNS vs SA
Instance: M-n101-k10
Optimal cost: 820.00
Initial cost: 835.00

Method          Cost         Gap (%)      Improvement (%) Vehicles  
----------------------------------------------------------------------
ALNS            835.00       1.83         0.00            10        
SA              835.00       1.83         0.00            10        

SA is better by 0.00 (0.00%)



In [32]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Convergence Comparison", "Final Costs")
)

# ALNS: recorded every 50 iterations
alns_iterations = [i * 50 for i in range(len(cost_history_alns))]
fig.add_trace(
    go.Scatter(
        x=alns_iterations,
        y=cost_history_alns,
        mode='lines',
        name='ALNS',
        line=dict(color='blue', width=2)
    ),
    row=1, col=1
)

# SA: recorded every 100 iterations
sa_iterations = [i * 100 for i in range(len(cost_history_sa))]
fig.add_trace(
    go.Scatter(
        x=sa_iterations,
        y=cost_history_sa,
        mode='lines',
        name='SA',
        line=dict(color='orange', width=2)
    ),
    row=1, col=1
)

if CONFIG['instance']['optimal_cost']:
    max_iter = max(max(alns_iterations), max(sa_iterations))
    fig.add_trace(
        go.Scatter(
            x=[0, max_iter],
            y=[CONFIG['instance']['optimal_cost'], CONFIG['instance']['optimal_cost']],
            mode='lines',
            name='Optimal',
            line=dict(color='green', width=2, dash='dash')
        ),
        row=1, col=1
    )

fig.add_trace(
    go.Bar(
        x=['Initial', 'ALNS', 'SA'],
        y=[initial_solution.cost, solution_alns.cost, solution_sa.cost],
        marker_color=['gray', 'blue', 'orange'],
        text=[f"{initial_solution.cost:.0f}", 
              f"{solution_alns.cost:.0f}", 
              f"{solution_sa.cost:.0f}"],
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Iterations", row=1, col=1)
fig.update_xaxes(title_text="Method", row=1, col=2)
fig.update_yaxes(title_text="Cost", row=1, col=1)
fig.update_yaxes(title_text="Cost", row=1, col=2)

fig.update_layout(
    title_text=f"ALNS vs SA - {CONFIG['instance']['name']}",
    height=400
)

fig.show()